In [19]:
!pip install pyomo==6.7.1

  Using cached Pyomo-6.7.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.7 kB)
Using cached Pyomo-6.7.1-cp310-cp310-macosx_11_0_arm64.whl (5.3 MB)
  Attempting uninstall: pyomo
    Found existing installation: Pyomo 5.6.9
    Uninstalling Pyomo-5.6.9:
      Successfully uninstalled Pyomo-5.6.9


In [5]:
from pathlib import Path

import pandas as pd

In [6]:
excel_file = Path("../data/ChallengeXHEC23022024.xlsx")

In [7]:
schedule = pd.read_excel(excel_file, sheet_name=0)
clients = pd.read_excel(excel_file, sheet_name=1)
caregivers = pd.read_excel(excel_file, sheet_name=2)

In [8]:
schedule.Prestation.unique()

array(['REPAS', 'TOILETTE', 'VIE SOCIALE', 'AIDE MENAGERE',
       'ADMINISTRATION', 'ACCOMPAGNEMENTS COURSES PA', 'FEMME DE MENAGE',
       'VISITE MEDICALE', 'FORMATION', "GARDE D'ENFANTS", 'COORDINATION',
       'HOMMES TOUTES MAINS'], dtype=object)

In [9]:
discard_list = [
    "ADMINISTRATION",
    "VISITE MEDICALE",
    "FORMATION",
    "COORDINATION",
    "HOMMES TOUTES MAINS",
]

In [10]:
sched_one_day = schedule.loc[schedule.Date == "2024-01-03", :]

In [11]:
sched_one_day.Prestation.value_counts()

Prestation
TOILETTE                      41
REPAS                         25
AIDE MENAGERE                 14
ACCOMPAGNEMENTS COURSES PA     4
ADMINISTRATION                 3
VIE SOCIALE                    3
Name: count, dtype: int64

In [12]:
sched_one_day = sched_one_day[~sched_one_day.Prestation.isin(discard_list)]

In [13]:
sched_one_day["idx"] = sched_one_day.index

In [14]:
from datetime import datetime

common_date = sched_one_day["Date"].iloc[0]
sched_one_day["Heure de fin"] = sched_one_day["Heure de fin"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Heure de début"] = sched_one_day["Heure de début"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Duration"] = (
    sched_one_day["Heure de fin"] - sched_one_day["Heure de début"]
)
sched_one_day["Duration"] = sched_one_day["Duration"].apply(
    lambda x: x.seconds // 60
)

In [15]:
sched_one_day

,ID Client,ID Intervenant,Date,Heure de début,Heure de fin,Prestation,idx,Duration
177,522099746,162858075,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:30:00,TOILETTE,177,60
178,87852633,776399834,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:15:00,TOILETTE,178,45
179,559277088,842621873,2024-01-03,2024-01-03 07:45:00,2024-01-03 08:15:00,TOILETTE,179,30
180,315244686,810259688,2024-01-03,2024-01-03 08:00:00,2024-01-03 08:45:00,TOILETTE,180,45
181,714782168,710283561,2024-01-03,2024-01-03 08:00:00,2024-01-03 09:00:00,TOILETTE,181,60
...,...,...,...,...,...,...,...,...
262,794098228,78005437,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,262,60
263,559475456,838320706,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:40:00,TOILETTE,263,45
264,714782168,810259688,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,264,60
265,803656603,838320706,2024-01-03,2024-01-03 19:50:00,2024-01-03 20:20:00,TOILETTE,265,30


In [16]:
caregivers

,ID Intervenant,Latitude,Longitude,Compétences,Permis,Véhicule personnel,Dispo / Indispo
0,838320706,48.738516,1.391971,"AIDE MENAGERE, REPAS, TOILETTE",Oui,Oui,"Indispo 01/01, 30/01, 31/01"
1,609468992,48.640555,1.232581,"TOILETTE, REPAS, VIE SOCIALE, AIDE MENAGERE",Oui,Oui,"Indispo Tous les mercredis + 13/01, 14/01, 27/..."
2,78012267,48.729206,1.371985,"HOMMES TOUTES MAINS, JARDINAGE",Oui,Oui,Dispo le 25/01
3,818696864,48.744702,1.357921,"REPAS, AIDE MENAGERE, ACCOMPAGNEMENTS COURSES,...",Oui,Oui,Indispo tous les samedis et dimanche
4,746414886,48.769455,1.197644,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,Indispo tous les mercredis + le 05/01
5,78005437,48.712418,1.356278,"AIDE MENAGERE, ADMINISTRATION, REPAS, TOILETTE...",Oui,Oui,Indispo tous les samedis et dimanche
6,213237245,48.763226,1.241120,"TOILETTE, REPAS, AIDE MENAGERE",Oui,Oui,"Indispo 01/01, 02/01, 03/01, 13/01, 14/01, 16/..."
7,813991780,48.692715,1.073195,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,"Indispo tous les mercredis, vendredi, samedi, ..."
8,856089133,48.645418,1.523077,"ACCOMPAGNEMENTS COURSES PA, REPAS, TOILETTE, A...",Non,Non,"Indispo 05/01, 09/01, 11/01 + du 18/01 au 31/0..."
9,700168298,48.758099,1.210611,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,"Dispo le 01/01, 30/01 et 31/01"


In [17]:
schedule

,ID Client,ID Intervenant,Date,Heure de début,Heure de fin,Prestation
0,559475456,162858075,2024-01-01,07:15:00,07:45:00,REPAS
1,559277088,162858075,2024-01-01,07:45:00,08:30:00,TOILETTE
2,87852633,78007018,2024-01-01,07:45:00,08:30:00,TOILETTE
3,243033408,810259688,2024-01-01,07:45:00,08:15:00,TOILETTE
4,814940942,710283561,2024-01-01,07:45:00,09:20:00,TOILETTE
...,...,...,...,...,...,...
2865,559277088,710283561,2024-01-31,19:00:00,19:15:00,REPAS
2866,714782168,810259688,2024-01-31,19:00:00,20:00:00,REPAS
2867,559475456,710283561,2024-01-31,19:15:00,20:00:00,TOILETTE
2868,803656603,854577575,2024-01-31,19:20:00,19:50:00,TOILETTE


In [18]:
# Import pyomo libraries
import pyomo.environ as pe
import pyomo.gdp as pyogdp

from itertools import product

In [19]:
class CareScheduler:
    def __init__(self, excel_file):
        """
        Read cargiver and session data into Pandas DataFrames
        Args:
            excel_file (str): path to data in excel format
        """
        try:
            clients = pd.read_excel(excel_file, sheet_name=1)
            sessions = pd.read_excel(excel_file, sheet_name=0)

            # NOTE: remove later
            sessions = sessions.loc[sessions.Date == "2024-01-03", :]
            sessions["idx"] = sessions.index

            # create duration
            common_date = sessions["Date"].iloc[0]
            sessions["Heure de fin"] = sessions["Heure de fin"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Heure de début"] = sessions["Heure de début"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Duration"] = (
                sessions["Heure de fin"] - sessions["Heure de début"]
            )
            sessions["Duration"] = sessions["Duration"].apply(
                lambda x: x.seconds // 60
            )

            # discard
            discard_list = [
                "ADMINISTRATION",
                "VISITE MEDICALE",
                "FORMATION",
                "COORDINATION",
                "HOMMES TOUTES MAINS",
            ]
            sessions = sessions[~sessions.Prestation.isin(discard_list)]

            sessions = sessions.merge(clients, how="left", on="ID Client")
            sessions = sessions.drop(columns="ID Intervenant")
            self.df_sessions = sessions
        except FileNotFoundError:
            print("Session data not found.")

        try:
            self.df_cargeivers = pd.read_excel(excel_file, sheet_name=2)
        except FileNotFoundError:
            print("Caregiver data not found")

        self.model = self.create_model()

    def _generate_case_durations(self):
        """
        Generate mapping of cases IDs to case time for the procedure
        Returns:
            (dict): dictionary with CaseID as key and median case time (mins) for procedure as value
        """
        return pd.Series(
            self.df_sessions["Duration"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_disjunctions(self):
        """Returns:
        disjunctions (list): list of tuples containing disjunctions
        """
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        disjunctions = []
        for (case1, case2, cargiver) in product(cases, cases, cargivers):
            if (case1 != case2) and (
                case2,
                case1,
                cargiver,
            ) not in disjunctions:
                disjunctions.append((case1, case2, cargiver))

        return disjunctions

    def create_model(self):
        model = pe.ConcreteModel()

        # List of case IDs in home care client needs list
        model.CASES = pe.Set(initialize=self.df_sessions["idx"].tolist())
        # List of potential caregiver IDs
        model.CAREGIVERS = pe.Set(
            initialize=self.df_cargeivers["ID Intervenant"].tolist()
        )
        # List of tasks - all possible (caseID, caregiverID) combination
        model.TASKS = pe.Set(
            initialize=model.CASES * model.CAREGIVERS, dimen=2
        )
        # The duration (expected case time) for each operation
        model.CASE_DURATION = pe.Param(
            model.CASES, initialize=self._generate_case_durations()
        )

        # Decision Variables
        ub = 1440  # minutes in a day
        model.M = pe.Param(initialize=1e3 * ub)  # big M
        # max_util = 0.85

        # Binary flag, 1 if case is assigned to session, 0 otherwise
        model.SESSION_ASSIGNED = pe.Var(model.TASKS, domain=pe.Binary)
        # Start time of a case
        model.CASE_START_TIME = pe.Var(
            model.TASKS, bounds=(0, ub), within=pe.PositiveIntegers
        )
        # Downtime of a caregiver caregiver
        model.DOWN_TIME = pe.Var(
            model.CAREGIVERS, bounds=(0, ub), within=pe.PositiveIntegers
        )
        # Session utilisation
        # model.UTILISATION = pe.Var(model.CAREGIVERS, bounds=(0, max_util), within=pe.PositiveReals)


        model.DISJUNCTIONS = pe.Set(initialize=self._generate_disjunctions(), dimen=3)

        # Objective
        def objective_function(model):
            return pe.summation(model.DOWN_TIME)
            # return sum([model.SESSION_ASSIGNED[case, session] for case in model.CASES for session in model.CAREGIVERS])
            # return min(model.UTILISATION.extract_values().values())

        model.OBJECTIVE = pe.Objective(
            rule=objective_function, sense=pe.minimize
        )

        # Constraints
        # TODO: get minimum working hours
        # Case start time must be after 8 AM
        # def case_start_time(model, case, session):
        #     return model.CASE_START_TIME[case, session] >= self.df_sessions["Heure de début"].min()
        # model.CASE_START = pe.Constraint(model.TASKS, rule=case_start_time)

        # TODO: create maximum working hours
        # # Case end time must be before general working hours 22 PM
        # def case_end_time(model, case, session):
        #     return model.CASE_START_TIME[case, session] + model.CASE_DURATION[case] <= self.df_sessions["Heure de fin"].max()
        # model.CASE_END_TIME = pe.Constraint(model.TASKS, rule=case_end_time)

        # Cases can be assigned to a maximum of one caregiver
        def session_assignment(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case, session)]
                        for session in model.CAREGIVERS
                    ]
                )
                <= 1
            )

        model.SESSION_ASSIGNMENT = pe.Constraint(
            model.CASES, rule=session_assignment
        )

        # Constraint 6: Utilisation is defined as fraction of a theatre session taken up by cases
        def theatre_util(model, session):
            return model.DOWN_TIME[session] == 1440 - sum([model.SESSION_ASSIGNED[case, session]*model.CASE_DURATION[case] for case in model.CASES])
        model.THEATRE_UTIL = pe.Constraint(model.CAREGIVERS, rule=theatre_util)

        # TODO: -> question 3 if someone has no work
        # def compare_utils(model, caregiver):
        #     return model.UTILISATION[caregiver] >= 0.13
        # model.ALL_MIN = pe.Constraint(model.CAREGIVERS, rule=compare_utils)


        # TODO: create bounding box deadlines
        # Constraint 4: Cases must be completed before their target deadline
        # def set_deadline_condition(model, case, session):
        #     return model.SESSION_DATES[session] <= model.CASE_DEADLINES[case] + ((1 - model.SESSION_ASSIGNED[case, session])*model.M)
        # model.APPLY_DEADLINE = pe.Constraint(model.TASKS, rule=set_deadline_condition)

        # Constraint 5: No two cases can overlap
        def no_case_overlap(model, case1, case2, caregiver):
            return [model.CASE_START_TIME[case1, caregiver] + model.CASE_DURATION[case1] <= model.CASE_START_TIME[case2, caregiver] + \
                    ((2 - model.SESSION_ASSIGNED[case1, caregiver] - model.SESSION_ASSIGNED[case2, caregiver])*model.M),
                    model.CASE_START_TIME[case2, caregiver] + model.CASE_DURATION[case2] <= model.CASE_START_TIME[case1, caregiver] + \
                    ((2 - model.SESSION_ASSIGNED[case1, caregiver] - model.SESSION_ASSIGNED[case2, caregiver])*model.M)]
        model.DISJUNCTIONS_RULE = pyogdp.Disjunction(model.DISJUNCTIONS, rule=no_case_overlap)

        pe.TransformationFactory("gdp.bigm").apply_to(model)

        return model

    def solve(self):
        # Create solver (synchronous solve)
        solver = pe.SolverFactory("cbc", executable="/opt/homebrew/bin/cbc")

        # Add solver parameters (time limit)
        options = {"seconds": 60}
        for key, value in options.items():
            solver.options[key] = value

        # Solve model (verbose)
        solver_results = solver.solve(self.model, tee=True)
        return solver_results

In [1]:
min(scheduler.model.UTILISATION.extract_values().values()) # .display()

NameError: name 'scheduler' is not defined

In [20]:
scheduler = CareScheduler(excel_file)

In [21]:
solver_results = scheduler.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.11 
Build Date: Oct 25 2023 

command line - /opt/homebrew/bin/cbc -seconds 60 -printingOptions all -import /var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/tmp6a34k7yz.pyomo.lp -stat=1 -solve -solu /var/folders/y_/2nw1qxt94ydbkmv5qzg8p4080000gn/T/tmp6a34k7yz.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 60
Option for printingOptions changed from normal to all
Presolve 179679 (-89784) rows, 93984 (-89784) columns and 902040 (-179568) elements
Statistics for presolved model
Original problem has 181680 integers (181656 of which binary)
Presolved problem has 91896 integers (91872 of which binary)
==== 93960 zero objective 2 different
93960 variables have objective of 0
24 variables have objective of 1
==== absolute objective values 2 different
93960 variables have objective of 0
24 variables have objective of 1
==== for integers 91872 zero objective 2 different
91872 variables have objective of 0
24 variables have objectiv

In [22]:
model = scheduler.model

In [27]:
model.DISJUNCTIONS.display() # .extract_values()

DISJUNCTIONS : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size  : Members
    None :     3 :    Any : 89784 : {(177, 178, 838320706), (177, 178, 609468992), (177, 178, 78012267), (177, 178, 818696864), (177, 178, 746414886), (177, 178, 78005437), (177, 178, 213237245), (177, 178, 813991780), (177, 178, 856089133), (177, 178, 700168298), (177, 178, 810259688), (177, 178, 78007018), (177, 178, 764144581), (177, 178, 888888888), (177, 178, 854577575), (177, 178, 480302361), (177, 178, 78007874), (177, 178, 842621873), (177, 178, 710283561), (177, 178, 1452747150), (177, 178, 162858075), (177, 178, 739888851), (177, 178, 648993440), (177, 178, 776399834), (177, 179, 838320706), (177, 179, 609468992), (177, 179, 78012267), (177, 179, 818696864), (177, 179, 746414886), (177, 179, 78005437), (177, 179, 213237245), (177, 179, 813991780), (177, 179, 856089133), (177, 179, 700168298), (177, 179, 810259688), (177, 179, 78007018), (177, 179, 764144581), (177, 179, 888888888)

In [38]:
actions = [ k for k, v in model.SESSION_ASSIGNED.extract_values().items() if v == 1 ]

In [40]:
actions[0]

(177, 609468992)

In [44]:
actions_df = pd.DataFrame(actions, columns=["idx", "Cargegiver_ID"])

In [45]:
temp = scheduler.df_sessions.copy()
temp.merge(actions_df, how="left", on="idx")

,ID Client,Date,Heure de début,Heure de fin,Prestation,idx,Duration,Latitude,Longitude,Cargegiver_ID
0,522099746,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:30:00,TOILETTE,177,60,48.729059,1.374504,609468992
1,87852633,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:15:00,TOILETTE,178,45,48.691944,1.498773,78012267
2,559277088,2024-01-03,2024-01-03 07:45:00,2024-01-03 08:15:00,TOILETTE,179,30,48.721052,1.375689,854577575
3,315244686,2024-01-03,2024-01-03 08:00:00,2024-01-03 08:45:00,TOILETTE,180,45,48.732218,1.363744,78012267
4,714782168,2024-01-03,2024-01-03 08:00:00,2024-01-03 09:00:00,TOILETTE,181,60,48.740158,1.369426,609468992
...,...,...,...,...,...,...,...,...,...,...
82,794098228,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,262,60,48.764898,1.203185,609468992
83,559475456,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:40:00,TOILETTE,263,45,48.721052,1.375689,78012267
84,714782168,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,264,60,48.740158,1.369426,609468992
85,803656603,2024-01-03,2024-01-03 19:50:00,2024-01-03 20:20:00,TOILETTE,265,30,48.723600,1.382000,78012267


In [46]:
# model.pprint()
# model.x.pprint()